## 잠재 요인 협업 필터링 (행렬분해 사용)

In [21]:
import pandas as pd
rating_data = pd.read_csv('ratings_small.csv')
movie_data = pd.read_csv('movies.csv')

rating_data.head(5)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
movie_data.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [23]:
rating_data.shape

(100004, 4)

In [24]:
rating_data.drop('timestamp',axis=1,inplace=True)
rating_data

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0
...,...,...,...
99999,671,6268,2.5
100000,671,6269,4.0
100001,671,6365,4.0
100002,671,6385,2.5


In [25]:
movie_data.drop('genres',axis=1,inplace=True)
movie_data

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)
...,...,...
9120,162672,Mohenjo Daro (2016)
9121,163056,Shin Godzilla (2016)
9122,163949,The Beatles: Eight Days a Week - The Touring Y...
9123,164977,The Gay Desperado (1936)


In [26]:
# 공통 컬럼 movieid기준으로 merge
rating_movie_data = pd.merge(rating_data,movie_data,on='movieId')
rating_movie_data.head(3)

,userId,movieId,rating,title
0,1,31,2.5,Dangerous Minds (1995)
1,7,31,3.0,Dangerous Minds (1995)
2,31,31,4.0,Dangerous Minds (1995)


In [27]:
rating_movie_data.shape

(100004, 4)

In [28]:
rating_movie_data = rating_movie_data.pivot_table('rating',index='userId',columns='title').fillna(0)
rating_movie_data.shape

(671, 9064)

In [29]:
rating_movie_data.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
movie_rating_data = rating_movie_data.values.T
movie_rating_data.shape


(9064, 671)

In [32]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=12)
matrix = svd.fit_transform(movie_rating_data)
matrix.shape

(9064, 12)

In [33]:
import numpy as np
corr = np.corrcoef(matrix)
corr.shape

(9064, 9064)

In [34]:
movie_title = rating_movie_data.columns
movie_title_list = list(movie_title)
input_movie = movie_title_list.index('Avatar (2009)')

In [35]:
corr_input_movie = corr[input_movie]
list(movie_title[(corr_input_movie >= 0.9)])[:50]

['(500) Days of Summer (2009)',
 '127 Hours (2010)',
 '16 Blocks (2006)',
 '2012 (2009)',
 '21 (2008)',
 '3 Idiots (2009)',
 '300 (2007)',
 '300: Rise of an Empire (2014)',
 'Abduction (2011)',
 'Adjustment Bureau, The (2011)',
 'Adventures of Tintin, The (2011)',
 'Alice in Wonderland (2010)',
 'Amazing Spider-Man, The (2012)',
 'Angels & Demons (2009)',
 'Avatar (2009)',
 'Avengers, The (2012)',
 'Band of Brothers (2001)',
 'Batman: The Dark Knight Returns, Part 1 (2012)',
 'Batman: The Dark Knight Returns, Part 2 (2013)',
 'Battlestar Galactica (2003)',
 'Beowulf (2007)',
 'Big Hero 6 (2014)',
 'Blind Side, The  (2009)',
 'Blood Diamond (2006)',
 'Body of Lies (2008)',
 'Bolt (2008)',
 'Book of Eli, The (2010)',
 'Bourne Ultimatum, The (2007)',
 'Brave (2012)',
 'Bucket List, The (2007)',
 'Buried (2010)',
 'Captain America: The First Avenger (2011)',
 'Captain America: The Winter Soldier (2014)',
 'Casino Royale (2006)',
 'Children of Men (2006)',
 'Chronicle (2012)',
 'Click (2006

## 개인유저가 본 영화 기반

In [36]:
df_ratings = pd.read_csv('ratings_small.csv')
df_movies = pd.read_csv('movies.csv')

In [37]:
df_user_movie_ratings = df_ratings.pivot(
    index='userId',
    columns='movieId',
    values='rating'
).fillna(0)
# 유저별 각 영화 평점

In [38]:
df_user_movie_ratings.head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 유저들이 평가한 영화들의 평균 평점 구함
# 그 값을 유저 영화평점 값에서 뺌

In [40]:
matrix = df_user_movie_ratings.to_numpy()

# 유저의 평균 평점
user_ratings_mean = np.mean(matrix, axis=1)

# 유저 평점에 유저 평균 평점을 뺀 것
matrix_user_mean = matrix - user_ratings_mean.reshape(-1,1)
matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [41]:
pd.DataFrame(matrix_user_mean,columns=df_user_movie_ratings.columns).head(3)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,...,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625,-0.005625
1,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,3.970770,...,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230,-0.029230
2,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,...,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075,-0.020075


In [46]:
from scipy.sparse.linalg import svds
# U행렬, sigma행렬 V 전치 행렬 반환
U, sigma, Vt = svds(matrix_user_mean, k = 12)

In [47]:
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(671, 12)
(12,)
(12, 9066)


In [48]:
# sigma 행렬은 0이 아닌 값만 1차원 행렬로 표현된 상태
# 0이 포함된 대칭행렬로 변환할 때는 numpy의 diag이용
sigma = np.diag(sigma)
sigma.shape

(12, 12)

In [51]:
sigma[2]

array([  0.        ,   0.        , 116.29714259,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ])

In [ ]:
# 1. 원본 user-movie 평점 행렬이 있었음
# 2. 이를 user의 평균 점수를 빼서 matrix_user_mean 이라는 행렬로 만듬
# 3. 2번의 값을 SVD를 적용해 U, Sigma, Vt 행렬을 구했음
# 4. Sigma 행렬은 현재 0이 포함이 되지 않은 값으로만 구성되어 있음. 이를 대칭행렬로 변환

In [52]:
# matrix_user_mean을 SVD를 적용해 분해를 한 상태
# 다시 원본 행렬로 복구
svd_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1,1)

# 아까 사용자 평균을 빼주었으니 여기서는 더해줌

In [53]:
df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns = df_user_movie_ratings.columns)
df_svd_preds.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.079686,0.021779,-0.013837,-0.005870,-0.028877,0.032371,0.000715,-0.004428,-0.005219,0.038195,...,-0.004324,-0.004352,0.010478,-0.004256,-0.003944,-0.005674,0.018157,-0.005575,-0.005297,-0.003766
1,1.428452,1.608841,0.529476,0.168278,0.520809,1.107473,0.529719,0.089376,0.296270,1.970031,...,0.013227,-0.002275,0.020680,-0.005245,-0.007644,-0.021019,0.031243,-0.000957,-0.000753,0.026901
2,0.977246,0.396971,0.000299,0.027444,0.021287,0.141458,-0.057134,0.031633,-0.012538,0.383576,...,0.002761,0.004907,-0.014190,-0.000251,-0.006007,-0.003189,-0.026916,0.014637,0.013287,-0.005741
3,1.870844,1.169993,0.252202,0.094831,-0.181713,-0.511953,-0.027820,-0.143080,0.013247,1.461694,...,0.026412,-0.027245,0.054681,0.018450,0.034544,-0.035740,0.088889,-0.019365,-0.017113,0.066559
4,1.182777,0.924903,0.075998,0.061505,0.602680,-0.159825,0.339925,0.081534,-0.079666,0.535018,...,-0.029124,-0.029357,0.009064,-0.029092,-0.030890,-0.057453,0.026344,-0.024027,-0.024614,-0.032752


In [ ]:
# 인자로 사용자 아이디, 영화 정보 테이블, 평점 테이블 등을 받음
# 유저 아이디에 SVD로 나온 결과의 영화 평점이 가장 높은 데이터 순으로 정렬
# 유저가 본 데이터를 제외
# 유저가 안 본 영화에서 평점이 높은 것을 추천

In [54]:
def recommend_movies(df_svd_preds, user_id, ori_movies_df,ori_ratings_df,num_recommendations=5):
    # index로 적용 되어있어서 user_id -1 해야함
    user_row_number = user_id - 1
    
    # 최종적으로 만든 pred_df에서 유저 index에 따라 영화 평점 순 정렬
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)
    
    # 원본 평점 데이터에서 user id 에 해당하는 데이터 뽑음
    user_data = ori_ratings_df[ori_ratings_df.userId == user_id]
    
    # 위에서 뽑은 user_data와 원본 영화 데이터 합침
    user_history = user_data.merge(ori_movies_df, on = 'movieId').sort_values(['rating'],ascending=False)
    
    # 원본 영화 데이터에서 사용자가 본 영화 데이터를 제외한 데이터
    recommendations = ori_movies_df[~ori_movies_df['movieId'].isin(user_history['movieId'])]
    
    # 유저의 영화 평점이 높은 순으로 정렬한 데이터와  recommendations 합침
    recommendations = recommendations.merge(pd.DataFrame(sorted_user_predictions).reset_index(), on='movieId')
    
    # 컬럼 이름 바꾸고 정렬 후 리턴
    recommendations = recommendations.rename(columns = {user_row_number: 'predictions'}).sort_values('predictions', ascending=False)
    
    return user_history, recommendations
    


In [55]:
already_rated, predictions = recommend_movies(df_svd_preds, 1, df_movies, df_ratings, 10)

In [56]:
already_rated.head

<bound method NDFrame.head of     userId  movieId  rating   timestamp   
4        1     1172     4.0  1260759205  \
13       1     2105     4.0  1260759139   
12       1     1953     4.0  1260759191   
8        1     1339     3.5  1260759125   
19       1     3671     3.0  1260759117   
1        1     1029     3.0  1260759179   
2        1     1061     3.0  1260759182   
14       1     2150     3.0  1260759194   
17       1     2455     2.5  1260759113   
0        1       31     2.5  1260759144   
10       1     1371     2.5  1260759135   
9        1     1343     2.0  1260759131   
7        1     1293     2.0  1260759148   
6        1     1287     2.0  1260759187   
5        1     1263     2.0  1260759151   
15       1     2193     2.0  1260759198   
16       1     2294     2.0  1260759108   
3        1     1129     2.0  1260759185   
11       1     1405     1.0  1260759203   
18       1     2968     1.0  1260759200   

                                             title   
4   Cinema P

In [57]:
predictions

,movieId,title,genres,predictions
1114,1387,Jaws (1975),Action|Horror,0.164121
965,1214,Alien (1979),Horror|Sci-Fi,0.159863
1001,1252,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller,0.155467
484,541,Blade Runner (1982),Action|Sci-Fi|Thriller,0.153000
1030,1282,Fantasia (1940),Animation|Children|Fantasy|Musical,0.150326
...,...,...,...,...
4997,7153,"Lord of the Rings: The Return of the King, The...",Action|Adventure|Drama|Fantasy,-0.106974
3834,4973,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance,-0.109088
4371,5952,"Lord of the Rings: The Two Towers, The (2002)",Adventure|Fantasy,-0.115702
2356,2959,Fight Club (1999),Action|Crime|Drama|Thriller,-0.121521
